In [15]:
%run ../../src/feature_selection_scripts/Feature_Selection_Bin_Cla_SRV.ipynb

['Seq', 'sTos', 'sTtl', 'Offset', 'sMeanPktSz', 'dMeanPktSz', 'AckDat', 'icmp', 'tcp', 'udp', 'ECO', 'FIN', 'INT', 'RST', 'Start']


In [16]:
import pickle
from sklearn.ensemble import VotingClassifier
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report


In [17]:
class TabNetClassifierWrapper(BaseEstimator, ClassifierMixin):
    def __init__(self, tabnet_model):
        self.tabnet_model = tabnet_model

    def fit(self, X, y):
        self.tabnet_model.fit(X, y)
        return self

    def predict(self, X):
        return self.tabnet_model.predict(X)

    def predict_proba(self, X):
        return self.tabnet_model.predict_proba(X)

In [18]:
with open('../AdaBoost/adaboost_clf.pkl', 'rb') as f:
    adaboost_clf = pickle.load(f)

with open('../Light_GBM/lgb_clf.pkl', 'rb') as f:
    lgb_clf = pickle.load(f)

with open('../TabNet_Classifier/tabnet_clf.pkl', 'rb') as f:
    tabnet_clf = pickle.load(f)

with open('../SVM/svm_clf.pkl', 'rb') as f:
    svm_clf = pickle.load(f)

with open('../CatBoost/catboost_clf.pkl', 'rb') as f:
    catboost_clf = pickle.load(f)

with open('../XGBoost/xgb_clf.pkl', 'rb') as f:
    xgb_clf = pickle.load(f)

In [19]:
X = df[selected_feature_labels].values
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [20]:
tabnet_wrapper = TabNetClassifierWrapper(tabnet_clf)

In [21]:
classifiers = [('AdaBoost', adaboost_clf), 
               ('XGBoost', xgb_clf),
               ('CatBoost', catboost_clf),
               ('SVM', svm_clf),
               ('LightGBM', lgb_clf)]

voting_clf = VotingClassifier(estimators=classifiers, voting='hard')

voting_clf.fit(X_train, y_train)

y_pred = voting_clf.predict(X_test)

/opt/homebrew/lib/python3.11/site-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Learning rate set to 0.189162
0:	learn: 0.2111311	total: 44.7ms	remaining: 44.6s
1:	learn: 0.0664737	total: 85.8ms	remaining: 42.8s
2:	learn: 0.0267207	total: 126ms	remaining: 41.8s
3:	learn: 0.0143874	total: 163ms	remaining: 40.6s
4:	learn: 0.0088684	total: 202ms	remaining: 40.2s
5:	learn: 0.0068808	total: 244ms	remaining: 40.5s
6:	learn: 0.0061003	total: 283ms	remaining: 40.1s
7:	learn: 0.0051481	total: 323ms	remaining: 40.1s
8:	learn: 0.0042929	total: 363ms	remaining: 39.9s
9:	learn: 0.0039921	total: 402ms	remaining: 39.8s
10:	learn: 0.0037242	total: 437ms	remaining: 39.3s
11:	learn: 0.0036427	total: 477ms	remaining: 39.3s
12:	learn: 0.0034173	total: 518ms	remaining: 39.3s
13:	learn: 0.0032929	total: 557ms	remaining: 39.2s
14:	learn: 0.0030571	total: 619ms	remaining: 40.7s
15:	learn: 0.0028141	total: 660ms	remaining: 40.6s
16:	learn: 0.0025549	total: 706ms	remaining: 40.8s
17:	learn: 0.0024895	total: 741ms	remaining: 40.4s
18:	learn: 0.0023903	total: 781ms	remaining: 40.3s
19:	learn

In [22]:
accuracy = accuracy_score(y_test, y_pred)
precision_benign = precision_score(y_test, y_pred, pos_label=0)
recall_benign = recall_score(y_test, y_pred, pos_label=0)
f1_benign = f1_score(y_test, y_pred, pos_label=0)
precision_malicious = precision_score(y_test, y_pred, pos_label=1)
recall_malicious = recall_score(y_test, y_pred, pos_label=1)
f1_malicious = f1_score(y_test, y_pred, pos_label=1)

# Print evaluation metrics
print("Accuracy:", accuracy)
print("precision_benign:", precision_benign)
print("recall_benign:", recall_benign)
print("f1_benign:", f1_benign)
print("precision_malicious:", precision_benign)
print("recall_malicious:", recall_benign)
print("f1_malicious:", f1_malicious)

Accuracy: 0.9996414155204574
precision_benign: 0.9993062636866652
recall_benign: 0.9997825814274366
f1_benign: 0.9995443658115514
precision_malicious: 0.9993062636866652
recall_malicious: 0.9997825814274366
f1_malicious: 0.9997043819277011
